# YOLO v1
### 简介
+ 将分类问题转化为回归问题（定位 + 分类 =》 回归得到类别和坐标）
+ 优点：速度快、更高的背景图片查准率、更强的泛化能力
+ 缺点：准确率和召回率低于 两步 检测系统

### Unified Detection（统一检测？）
+ 将输入图像分成 S * S个 grid（栅格），如果一个物体的中心掉落在一个 grid cell （网格单元）内，那么这个 grid cell （网格单元）就负责检测这个物体。
+ 每一个 grid cell 预测 B 个 bounding boxes，以及这些 bounding boxes 的得分：score，
    + 所以confidence = P(Object) * IOU(truth, pred)
        + 若bounding box包含物体，则P(object) = 1；否则P(object) = 0
        + IOU为预测bounding box与物体真实区域的交集面积（以像素为单位，用真实区域的像素面积归一化到[0,1]区间）
+ 每个边框由五个预测值组成：x,y,w,h,confidence。
    + 坐标（x,y）代表了 bounding box 的中心与 grid cell 边界的相对值。
    + width、height 则是相对于整幅图像的预测值(边框的宽和高)。
    + confidence 就是 IoU 值
    + 实际训练过程中，w和h的值使用图像的宽度和高度进行归一化到[0,1]区间内；x，y是bounding box中心位置相对于当前格子位置的偏移值，并且被归一化到[0,1]。
+ 每一个栅格还要预测C个 conditional class probability（条件类别概率）：
    + Pr(Class|Object)。即在一个栅格包含一个Object的前提下，它属于某个类的概率。我们只为每个栅格预测一组（C个）类概率，而不考虑框B的数量。
    + 注：
        + conditional class probability信息是针对每个网格的。
        + confidence信息是针对每个bounding box的。
+ 注：
    + 由于输出层为全连接层，因此在检测时，YOLO训练模型只支持与训练图像相同地输入分辨率。
    + 虽然每个格子可以预测B个bounding box，但是最终只选择只选择IOU最高的bounding box作为物体检测输出，即每个格子最多只预测出一个物体。当物体占画面比例较小，如图像中包含畜群或鸟群时，每个格子包含多个物体，但却只能检测出其中一个。这是YOLO方法的一个缺陷。

### 网络设计
+ 主体是24层卷积网络，末端采用2层全连接+reshape保证输出shape=（7，7，30）
+ 模型中使用的激活函数都是leaky ReLU（alpha = 0.1）

### 训练
+ 预训练分类网络：前20个卷机网络+average-pooling layer（平均池化层）+ fully connected layer（全连接层）  输入：224*224
+ 训练检测网络：在预训练网络中增加卷积和全链接层可以改善性能。在作者的例子基础上添加4个卷积层和2个全链接层，随机初始化权重。检测要求细粒度的视觉信息，所以把网络输入把224\*224变成448\*448。

### 损失函数
+ 使用SSE作为loss（和方差，即所有元素详减取平方，然后累加。区别MSE）
+ 由于多数是不包含物体的，即多数的confidence=0，从而这部分的梯度会很高，需要减小权重
    + 论文中设置lambda(noobj) = 0.5,lambda(coord) = 5
    + 由于大的bbox得分会高于小的bbox，所以这里不直接使用w和h，而是使用sqrt(w), sqrt(h)
+ loss公式
![loss公式](https://pic3.zhimg.com/80/v2-67ef186c0bee9d774beae6291e8f48e6_720w.jpg)
    + 其中：
        + s=7：输入图片平均切分为49个小图块对待
        + B=2：每个图块最多预测2个gt_box
        + 1(obj, ij)：如果第 i 个图块的第 j 个预测的bbox，如果存在1，否则0
        + 1(noobj, ij) ：如果第 i 个图块的第 j 个预测的bbox，如果存在0，否则1
        + 1(obj, i) ：如果存在任意物体，那么为1，否则为0
        + 第三行：存在任意物体才计算它关于该类别的概率的SE，不包含物体的小图块不考虑分类误差

### 训练
+ 学习率策略：
    + 第1个epoch：学习率逐渐上升方法（从1e-3到1e-2缓慢上升）。如果从一开始就使用高的学习率，那么模型会发散；
    + 接下来75个epoch，lr = 1e-2
    + 接下来30个epoch，lr = 1e-3
    + 接下来30个epoch，lr = 1e-4
+ 避免过拟合方法：
    + dropout：第一个全连接后面使用0.5 dropout
    + 数据集增强：随机缩放（0.2）、随机曝光和饱和度